In [ ]:
from Tools import *
from photoObject import *
from MCMCObject import *
import emcee
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from Tools import *
from joblib import Parallel, delayed
import xmatch
from astropy.table import Table
from astropy.io import ascii

In [ ]:
Red=['z6_GND_8934',
 'z5_GND_10574',
 'z6_GNW_11480',
 'z7_GNW_22375',
 'z6_GND_43953',
 'z5_GNW_29609',
 'z5_GND_6005',
 'z7_GNW_7284']


Red2=['z5_GNW_15423', 'z5_GND_44499', 'z6_GNW_32543', 'z6_GNW_2045', 'z6_GNW_6554', 'z6_GNW_16141', 'z6_GNW_14478', 'z6_GND_34516', 'z6_GND_44248', 'z7_GNW_22375', 'z6_GND_29625', 'z5_GNW_29609', 'z5_GND_908', 'z5_GND_6005', 'z7_GNW_23317', 'z7_GNW_7284', 'z7_GND_18323', 'z7_GND_20984', 'Stark11_41_13100']

Red2 = [item for item in Red2 if item not in Red]
print(Red2)

Blue=['z6_GNW_23437',
 'z6_GNW_10822',
 'z6_GND_29564',
 'z6_GND_41463',
 'z6_GNW_18532',
 'z7_GND_8358',
 'z5_GNW_25408',
 'z5_GNW_32760',
 'z5_GNW_23799',
 'z5_GNW_22490',
 'z5_GNW_21283',
 'z5_GNW_8402',
 'z6_GNW_10897',
 'z5_GND_27819',
 'z5_GNW_17295',
 'Stark11_43_3982',
 'Stark11_32_16773',
 'Jung18_z6_GND_28438',
 'Stark11_35_22381',
 'Stark11_42_6706']

highEW=['z6_GND_29625']

## Run MCMC code over all galaxies in CANDELS , Jades and 3dhst


## Jades

In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")
#mask=Catalog["slope"]<-3
#Catalog=Catalog[mask]

In [ ]:
import xmatch
Jades=fits.open("../A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix_phot_apcorr.fits")

idx,d2d=xmatch.xmatch(Catalog["ra_candels"],Catalog["dec_candels"],Jades[1].data["RA"],Jades[1].data["DEC"])
mask=[]
for i,d in zip(idx,d2d):
    if d.value<(0.3/3600.):
        mask.append(True)
    else:
        mask.append(False)

CatJades=Catalog[mask]
idx=idx[mask]
d2d=d2d[mask]


In [ ]:
plt.hist(Catalog["WFC3_F160W_FLUX"],bins=20)
print(np.nanmin(Catalog["WFC3_F160W_FLUX"]))
plt.show()

In [ ]:
Jades=fits.open("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix_phot_apcorr.fits")
j=fits.open("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix.eazypy.zout.fits")
Filters=np.array(["f435w_tot_2","f435w_etot_2","f606w_tot_2","f606w_etot_2","f775w_tot_2","f775w_etot_2","f814w_tot_2","f814w_etot_2","f850lp_tot_2","f850lp_etot_2","f105w_tot_2","f105w_etot_2","f110w_tot_2","f110w_etot_2","f125w_tot_2","f125w_etot_2","f140w_tot_2","f140w_etot_2","f160w_tot_2","f160w_etot_2",
                  "f090w_tot_2",'f090w_etot_2',"f115w_tot_2",'f115w_etot_2',"f150w_tot_2",'f150w_etot_2',"f200w_tot_2",'f200w_etot_2',"f277w_tot_2",'f277w_etot_2',"f335m_tot_2",'f335m_etot_2',"f356w_tot_2",'f356w_etot_2',"f410m_tot_2",'f410m_etot_2',"f444w_tot_2",'f444w_etot_2'])

DataJades={}

mask = (Jades[1].data["f160w_tot_2"]>0.036) & (Jades[1].data["f160w_tot_2"]<0.7) & (Jades[1].data["f160w_tot_2"]/Jades[1].data["f160w_etot_2"]>3) 
JadesD=Jades[1].data[mask]
for i in range(0,len(JadesD)):
    fluxs=[JadesD[i][f] for f in Filters]
    mags=[]
    count=0
    for f,n in zip(fluxs,Filters):
        if "_e" not in n:
            if fluxs[count+1]<0:
                mags.append(np.nan)
            else:
                mags.append(Tools.Flux2MagAB(f*1e-6))
                prevmag=Tools.Flux2MagAB(f*1e-6)
                prev=f


        if "_e" in n:
            if f<0:
                mags.append(np.nan)
            else:
                magP=Tools.Flux2MagAB((prev+f)*1e-6)
                magM=Tools.Flux2MagAB((prev-f)*1e-6)
                mags.append(np.nanmean([magM-prevmag,prevmag-magP]))


        count+=1
    #mags=[Tools.Flux2MagAB(f*1e-6) for f in fluxs ]
    DataJades[JadesD[i]["id"]]=mags



idx,d2d=xmatch.xmatch([J["ra"] for J in JadesD],[J["dec"] for J in JadesD],j[1].data["ra"],j[1].data["dec"])
mask=[]
for i,d in zip(idx,d2d):
    if d.value<(0.2/3600.):
        mask.append(True)
    else:
        mask.append(False)

tempDic={}
for i in range(0,len(DataJades.keys())):
    if mask[i]==True and ( (j[1].data[idx[i]]["z_spec"]>5  and (j[1].data[idx[i]]["z_spec"]<6.5  ) or (j[1].data[idx[i]]["z_phot"]>5) and j[1].data[idx[i]]["z_phot"]<6.5)  ):
        Photom=DataJades[list(DataJades.keys())[i]]
        Photom.append([j[1].data[idx[i]]["z_spec"],j[1].data[idx[i]]["z_phot"]])
        tempDic[list(DataJades.keys())[i]]=Photom
DataJades=tempDic

FiltNames   =  [name.replace("flux","mag") for name in Filters]
FiltErrNames=  [name.replace("flux","magerr") for name in Filters]
#Filts=[]
#for i in range(0,len(FiltNames)):
    #Filts.append(FiltNames[i])
    #Filts.append(FiltErrNames[i])

tempList=[]
for k in DataJades.keys():
    tempDic={}
    tempDic["ID"]=k
    for i,key in enumerate(FiltNames):
        tempDic[key]=DataJades[k][i]

    specz=DataJades[k][-1][0]
    if specz==-1:
        specz=np.nan
    tempDic["zsys"]=specz
    tempDic["z"]=np.nan
    tempDic["photoz"]=DataJades[k][-1][1]
    tempList.append(tempDic)

table = Table(tempList)

In [ ]:

%load_ext autoreload
%autoreload 

PObj= np.array([MCMCObject(c) for c in table])

Width=[821.68,1771.88,1379.43,1888.66,1321.98,2371.97 ,3856.85,2674.40,3569.86,2750.15,
        1772.74,2055.13,2890.43,4190.39,6448.30,3389.42,7239.30	,4262.86,10676.00]

Cwaves  = [ 4360.06 , 6035.73, 7757, 8129.21, 9080.26,10542.47, 11696.34 , 12503.91 , 13983.36 , 15437.71
    ,8984, 11433.62,	14872.56,19680.41,27278.58,	33537.23,35287.04,40723.18,43504.26	]

EffW = [[Cwave - width, Cwave + width] for Cwave, width in zip(Cwaves, Width)]
#EffW    = [ [3610.23,4883.77	],[	4634.30,7180.10]   ,[7000,8500]  , [6869.59,9632.01	]  ,[8007.01,10862.13], [8955.2,12130.5] , [8845.,14122.]   , [10853.22, 14141.73] , [11864.94,16133.14] , [13857.70,17003.09]
#    ,[8098.0,9870], [10406.1,12461.1] ,[13427.345,16317.775],[17585.215,21775.605], [23971.275,30585.885],[31842.52,35231.94],[31667.39,38906.69],[38591.75,42854.61],[38166.26,48842.26]]# , 1098    , 3856     , 1232  ]


Filters=np.array(["f435w_tot_2","f606w_tot_2","f775w_tot_2","f814w_tot_2","f850lp_tot_2","f105w_tot_2","f110w_tot_2","f125w_tot_2","f140w_tot_2","f160w_tot_2",
                  "f090w_tot_2","f115w_tot_2","f150w_tot_2","f200w_tot_2","f277w_tot_2","f335m_tot_2","f356w_tot_2","f410m_tot_2","f444w_tot_2"])
FiltNames   =  [name.replace("tot","tot") for name in Filters]
FiltErrNames=  [name.replace("tot","etot") for name in Filters]


[gal.setFilters(FiltNames,FiltErrNames) for gal in PObj]
[gal.setCWave(Cwaves) for gal in PObj]
[gal.setEffWidth(EffW) for gal in PObj]
RUN_MCMC=True #Put True if we want to run it
def runCode(p,i=0):
    slopeMuvAllJades =   np.array(p.calculateUVslope(ShowPlots=True)) #If dont want plotting, showplots=False
    #np.save("SlopeMuv_MCMC.npy",slopeMuv)
    return slopeMuvAllJades

#num=[0,"ALL"]
if RUN_MCMC==True:
    # If you want to see the plots, dont run it parallelized
    #slopeMuvAllJades=Parallel(n_jobs=4)(delayed(runCode)(p,i) for i,p in enumerate(PObj[:]))#         Paralellized
    slopeMuvAllJades=[runCode(gal) for gal in PObj[:]]  

#np.save("../A-Catalogs/Muv-Beta-EW/slopeMuvAllJades.npy",slopeMuvAllJades)

In [ ]:
slopeMuvAllJades=np.load("../A-Catalogs/Muv-Beta-EW/slopeMuvAllJades.npy",allow_pickle=True)

In [ ]:
beta=[]
dbeta=[]
muv=[]
dmuv=[]

for gal in slopeMuvAllJades:
    beta.append(gal[0][0])
    muv.append(gal[1][0])
    dbeta.append(gal[0][1])
    dmuv.append(gal[1][1])
beta=np.array(beta)
muv=np.array(muv)
dbeta=np.array(dbeta)
dmuv=np.array(dmuv)

In [ ]:
mask=dbeta<1
beta=beta[mask]
muv=muv[mask]
dbeta=dbeta[mask]
dmuv=dmuv[mask]

mask=muv>-20
beta=beta[mask]
muv=muv[mask]
dbeta=dbeta[mask]
dmuv=dmuv[mask]

In [ ]:
print(np.nanmedian(beta))

Catalog=Catalog[Catalog["Muv"]>-20]
print(np.nanmedian(Catalog["slope"]))

In [ ]:
plt.hist(Catalog["f115w_tot_2"])

In [ ]:
np.nanmin(Catalog["f115w_tot_2"])

In [ ]:

plt.errorbar(muv,beta,xerr=dmuv,yerr=dbeta,fmt="o",alpha=0.4,zorder=0,label="Subsample of JADES-DJA")
plt.errorbar(Catalog["Muv"],Catalog["slope"],xerr=Catalog["dMuv"],yerr=Catalog["dslope"],fmt="o",alpha=0.7,ms=5,label="All our data at Muv>-20")
plt.axhline(y=-2.2,color="blue",ls="--")
plt.axhline(y=-1.8,color="orange",ls="--")
plt.xlim(-20.4,-18)
plt.xlabel("Muv")
plt.ylabel("Beta")
plt.legend()
plt.show()

In [ ]:
plt.errorbar(muv,beta,xerr=dmuv,yerr=dbeta,fmt="o",errorevery=10)



In [ ]:
plt.errorbar(muv[mask],beta[mask],xerr=dmuv[mask],yerr=dbeta[mask],fmt="o")

## CANDELS

In [ ]:
CANDELS=fits.open("../A-Catalogs/CandlesShards.fit")
CANDELSz=fits.open("../A-Catalogs/CandelsShards_photoz.fit")

def microjanskys_to_abmag(microjanskys):
    return - 2.5 * np.log10(microjanskys / 1e6)+8.9

Filters=["FU-KPNO","e_FU-KPNO","FU-LBC","e_FU-LBC","F435W","e_F435W","F606W","e_F606W","F775W","e_F775W","F814W","e_F814W","F850LP","e_F850LP","F105W","e_F105W","F125W","e_F125W","F140W","e_F140W","F160W","e_F160W"]
        
DataCandels={}
CandelsD=random.choices(CANDELS[1].data,k=10000)

for i in range(0,len(CandelsD)):
    fluxs=[CandelsD[i][f] for f in Filters]
    mags=[]
    count=0
    for f,n in zip(fluxs,Filters):
        if "e_" not in n:
            if fluxs[count+1]==0.0:
                mags.append(np.nan)
            else:
                mags.append(microjanskys_to_abmag(f))
                prevmag=microjanskys_to_abmag(f)
                prev=f

        if "e_" in n:
            if f==0.0:
                mags.append(np.nan)
            else:
                magP=microjanskys_to_abmag((prev+f))
                magM=microjanskys_to_abmag((prev-f))
                mags.append(np.nanmean([magM-prevmag,prevmag-magP]))

        count+=1
    #mags=[Tools.Flux2MagAB(f*1e-6) for f in fluxs ]
    DataCandels[CandelsD[i]["id"]]=mags





tempDic={}
zcat_ID=CANDELSz[1].data["ID"]
for i,keyw in enumerate(DataCandels.keys()):
    index = np.where(zcat_ID == int(keyw))[0][0]
    z=CANDELSz[1].data["zbest"][index]
    if z>5 and z<6.5:
        Photom=DataCandels[list(DataCandels.keys())[i]]
        Photom.append([np.nan,z])
        tempDic[list(DataCandels.keys())[i]]=Photom
DataCandels=tempDic

In [ ]:

len(DataCandels)

In [ ]:
CANDELSz[1].header

### Fix Hu10_z7_1 flux

In [ ]:
id="Hu10_z7_1"
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")


flux=np.log10(4.4604980978190176e-17)
eflux=np.log10(3.189218850008286e-18)

for gal in Catalog:
    if gal["id_charlotte"]==id:
        gal["LyaFlux"]=flux
        gal["LyaErr"]=eflux
#ascii.write(Catalog, '../A-Catalogs/Binospec-Candels.cat', overwrite=True)  


In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")


## Add Muv to Hu10_z6_3

it did not have photometry in any catalog, and only one useful band on Hu paper


In [ ]:
from astropy.io import ascii
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from scipy.optimize import minimize
import math
import scipy.special as scispe
from random import Random
import random
from scipy.interpolate import interp1d
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")

z=5.635
Mab=25.99
flux = 10**((Mab + 48.6) / -2.5)
cosmo   =   FlatLambdaCDM(H0=70,Om0=0.3,Tcmb0=2.725)


flux    =   (flux)/(1+z)
m       =   -2.5*np.log10(flux) -48.6
dL      =   cosmo.luminosity_distance(z).value
Mabs    =   m-5*(np.log10(dL*10**6) - 1)



id="Hu10_z6_3"
for gal in Catalog:
    if gal["id_charlotte"]==id:
        gal["Muv"]=Mabs
        gal["dMuv"]=0.2
        print(gal)
#ascii.write(Catalog, '../A-Catalogs/Binospec-Candels.cat', overwrite=True)  


## CANDLES/SHARDS HST

In [ ]:
highEW=["z6_GNW_23437"]

In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")
mask = [gal["id_charlotte"] in highEW for gal in Catalog]
Catalog=Catalog[mask]
print(Catalog)
PObj= np.array([MCMCObject(c) for c in Catalog])
Cwaves  = [ 	4360.06  , 6035.73 , 7730.77 ,8129.21,9080.26,10542.47 ,  12503.91 , 13983.36 , 15437.71 ]# , 5413.32 , 11696.34 , 6323.81   ]
#EffW    = [ [3610.23,4883.77	],[	4634.30,7180.10]    , [6803.72,8631.82]   , [6869.59,9632.01	]  ,[8007.01,10862.13], [8955.2,12130.5] ,  [10853.22, 14141.73] , [11864.94,16133.14] , [13857.70,17003.09] ]# , 1098    , 3856     , 1232  ]
Width=[821.68,1771.88,1379.43,1888.66,1321.98,2371.97 ,2674.40,3569.86,2750.15]

EffW = [[Cwave - width, Cwave + width] for Cwave, width in zip(Cwaves, Width)]

flts=["ACS_F435W","ACS_F606W","ACS_F775W","ACS_F814W","ACS_F850LP","WFC3_F105W","WFC3_F125W","WFC3_F140W","WFC3_F160W"]#,"555","110","625"] 
flt=["","_FLUX"]
flterr=["","_FLUXERR"]
FiltNames   =  []
FiltErrNames=  []
for i in flts:
    FiltNames.append(flt[0]+i+flt[1])
    FiltErrNames.append(flterr[0]+i+flterr[1])

[gal.setFilters(FiltNames,FiltErrNames) for gal in PObj]
[gal.setCWave(Cwaves) for gal in PObj]
[gal.setEffWidth(EffW) for gal in PObj]

#####

RUN_MCMC=True#Put True if we want to run it

def runCode(p,i=0):
    print(p.cat["id_charlotte"],p.cat["slope"],p.cat["photoz"])
    slopeMuv =   np.array(p.calculateUVslope(ShowPlots=True)) #If dont want plotting, showplots=False
    #np.save("SlopeMuv_MCMC.npy",slopeMuv)
    return slopeMuv

#num=[0,"ALL"]
if RUN_MCMC==True:
    # If you want to see the plots, dont run it parallelized
    #slopeMuvCANDLES=Parallel(n_jobs=4)(delayed(runCode)(p,i) for i,p in enumerate(PObj[:]))#         Paralellized
    slopeMuvCANDLES=[runCode(gal) for gal in PObj[:]]    
    #np.save("../A-Catalogs/Muv-Beta-EW/slopeMUVCANDLES.npy",slopeMuvCANDLES)                                 # Not parallelized, allows plots
    

if RUN_MCMC==False: 
    slopeMuvCANDLES=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUVCANDLES.npy",allow_pickle=True)



## JADES NIRCam

Here I will :
- match to 0.5 arcsec
- Add to my catalog
- Run MCMC
- Compare to old MCMC

In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")


In [ ]:
Jades=fits.open("../A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix_phot_apcorr.fits")

In [ ]:
%load_ext autoreload
%autoreload 

import xmatch
Jades=fits.open("../A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix_phot_apcorr.fits")

idx,d2d=xmatch.xmatch(Catalog["ra_candels"],Catalog["dec_candels"],Jades[1].data["RA"],Jades[1].data["DEC"])
mask=[]
for i,d in zip(idx,d2d):
    if d.value<(0.3/3600.):
        mask.append(True)
    else:
        mask.append(False)

CatJades=Catalog[mask]
idx=idx[mask]
d2d=d2d[mask]


Filters=np.array(["f435w_tot_2","f435w_etot_2","f606w_tot_2","f606w_etot_2","f775w_tot_2","f775w_etot_2","f814w_tot_2","f814w_etot_2","f850lp_tot_2","f850lp_etot_2","f105w_tot_2","f105w_etot_2","f110w_tot_2","f110w_etot_2","f125w_tot_2","f125w_etot_2","f140w_tot_2","f140w_etot_2","f160w_tot_2","f160w_etot_2",
                  "f090w_tot_2",'f090w_etot_2',"f115w_tot_2",'f115w_etot_2',"f150w_tot_2",'f150w_etot_2',"f200w_tot_2",'f200w_etot_2',"f277w_tot_2",'f277w_etot_2',"f335m_tot_2",'f335m_etot_2',"f356w_tot_2",'f356w_etot_2',"f410m_tot_2",'f410m_etot_2',"f444w_tot_2",'f444w_etot_2'])

for name in Filters:
    Catalog[name] = np.nan
Data={}
for i in range(0,len(CatJades)):
    IDX=idx[i]
    fluxs=[Jades[1].data[IDX][f] for f in Filters]
    mags=[]
    is_nan=False
    for f,n in zip(fluxs,Filters):
    
        if "e" in n:
            if is_nan:
                mags.append(np.nan)
            else:
                magP=Tools.Flux2MagAB((prev+f)*1e-6)
                magM=Tools.Flux2MagAB((prev-f)*1e-6)
                mags.append(np.mean([magM-prevmag,prevmag-magP]))
        else:
            mags.append(Tools.Flux2MagAB(f*1e-6))
            prev=f
            prevmag=Tools.Flux2MagAB(f*1e-6)
            if np.isnan(prev):
                is_nan=True

        
    
    
    #mags=[Tools.Flux2MagAB(f*1e-6) for f in fluxs ]
    Data[CatJades[i]["id_charlotte"]]=mags




#Data["z7_GNW_24443"][5]=0.05
#Data["z5_GNW_23799"][5]=0.05
#Data["Hu10_z6_6"][5]=0.05
#Data["Hu10_z7_2"][5]=0.05

EmptyArray=[np.nan for gal in Catalog]

# Ensure that if an even element is np.nan, the previous element is also np.nan
for key, values in Data.items():
    for i in range(1, len(values), 2):
        if np.isnan(values[i]):
            values[i-1] = np.nan





for i,name in enumerate(Filters):
    name=name.replace("flux","mag")
    Catalog[name]=EmptyArray
    for gal in Catalog:
        try:
            #print(Data[gal["id_charlotte"]])
            gal[name]=Data[gal["id_charlotte"]][i]
        except:
            continue
            #print("No Jades Data")
#ascii.write(Catalog, '../A-Catalogs/Binospec-Candels.cat', overwrite=True)  


### Add Muv and Slope from JADES or 3DHST-DjA

- I have photometry from JADES JWST for some sources
- For other ones I have HST data that has the same reduction pipeline
- For a couple they are not in DjA, so i will have to use Candels/Shards 


In [ ]:
#Mask for ones that have Jades

import xmatch
Jades=fits.open("../A-Catalogs/data/large_files/MoreFresco/gdn-grizli-v7.3-fix_phot_apcorr.fits")
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")

idx,d2d=xmatch.xmatch(Catalog["ra_candels"],Catalog["dec_candels"],Jades[1].data["RA"],Jades[1].data["DEC"])
maskJades=[]
for i,d in zip(idx,d2d):
    if d.value<(0.3/3600.):
        maskJades.append(True)
    else:
        maskJades.append(False)

F160W_Jades=np.array([Tools.Flux2MagAB(Jades[1].data[i]["f160w_flux_aper_2"]*1e-6) for i in idx])
F160W_test_Jades=np.array([Tools.Flux2MagAB(Jades[1].data[i]["f160w_tot_2"]*1e-6) for i in idx])


In [ ]:
f3DHST=ascii.read("../A-Catalogs/goodsn_3dhst.v4.1.cat")


idx,d2d=xmatch.xmatch(Catalog["ra_candels"],Catalog["dec_candels"],f3DHST["ra"],f3DHST["dec"])
mask3DHST=[]
for i,d in zip(idx,d2d):
    if d.value<(0.3/3600.):
        mask3DHST.append(True)
    else:
        mask3DHST.append(False)

F160W_3DHST=np.array([Tools.Flux2MagAB_Gabe(f3DHST[i]["faper_F160W"]) for i in idx])
F160W_test_3DHST=np.array([Tools.Flux2MagAB_Gabe(f3DHST[i]["f_F160W"]) for i in idx])

In [ ]:
plt.scatter(F160W_test_Jades[maskJades],F160W_test_3DHST[maskJades])
plt.plot([23,28],[23,28],ls="-")



plt.xlabel("mag(F160W) JADES,  [f160w_tot_2]")
plt.ylabel("mag(F160W) 3DHST,  [f_F160W]")


### Calculate Muv and Beta with JADES

Use tot_2

In [ ]:
Catalog[Catalog["zsys"]>0]

In [ ]:
%load_ext autoreload
%autoreload 

Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")
Catalog=Catalog[Catalog["zsys"]>0]
#Catalog=Catalog[mask]
PObj= np.array([MCMCObject(c) for c in Catalog])


Width=[821.68,1771.88,1379.43,1888.66,1321.98,2371.97 ,3856.85,2674.40,3569.86,2750.15,
        1772.74,2055.13,2890.43,4190.39,6448.30,3389.42,7239.30	,4262.86,10676.00]

Cwaves  = [ 4360.06 , 6035.73, 7757, 8129.21, 9080.26,10542.47, 11696.34 , 12503.91 , 13983.36 , 15437.71
    ,8984, 11433.62,	14872.56,19680.41,27278.58,	33537.23,35287.04,40723.18,43504.26	]

EffW = [[Cwave - width, Cwave + width] for Cwave, width in zip(Cwaves, Width)]
#EffW    = [ [3610.23,4883.77	],[	4634.30,7180.10]   ,[7000,8500]  , [6869.59,9632.01	]  ,[8007.01,10862.13], [8955.2,12130.5] , [8845.,14122.]   , [10853.22, 14141.73] , [11864.94,16133.14] , [13857.70,17003.09]
#    ,[8098.0,9870], [10406.1,12461.1] ,[13427.345,16317.775],[17585.215,21775.605], [23971.275,30585.885],[31842.52,35231.94],[31667.39,38906.69],[38591.75,42854.61],[38166.26,48842.26]]# , 1098    , 3856     , 1232  ]


Filters=np.array(["f435w_tot_2","f606w_tot_2","f775w_tot_2","f814w_tot_2","f850lp_tot_2","f105w_tot_2","f110w_tot_2","f125w_tot_2","f140w_tot_2","f160w_tot_2",
                  "f090w_tot_2","f115w_tot_2","f150w_tot_2","f200w_tot_2","f277w_tot_2","f335m_tot_2","f356w_tot_2","f410m_tot_2","f444w_tot_2"])
FiltNames   =  [name.replace("tot","tot") for name in Filters]
FiltErrNames=  [name.replace("tot","etot") for name in Filters]


[gal.setFilters(FiltNames,FiltErrNames) for gal in PObj]
[gal.setCWave(Cwaves) for gal in PObj]
[gal.setEffWidth(EffW) for gal in PObj]
RUN_MCMC=True#Put True if we want to run it
def runCode(p,i=0):
    print(p.cat)
    slopeMuv =   np.array(p.calculateUVslope(ShowPlots=False)) #If dont want plotting, showplots=False
    print(p.type)
    #np.save("SlopeMuv_MCMC.npy",slopeMuv)
    return slopeMuv

#num=[0,"ALL"]
if RUN_MCMC==True:
    # If you want to see the plots, dont run it parallelized
    #slopeMuvJADES=Parallel(n_jobs=4)(delayed(runCode)(p,i) for i,p in enumerate(PObj[:]))#         Paralellized
    slopeMuvJADES_Zphot=[runCode(gal) for gal in PObj[:]]      
    #np.save("../A-Catalogs/Muv-Beta-EW/slopeMUVJades_Nov1.npy",slopeMuvJADES)

else:
    slopeMuvJADES=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUVJades_zspec.npy",allow_pickle=True)

In [12]:
slopeMuvJADES_Zspec

[array([list([-1.5898790524595179, 0.15703965862747027]),
        list([-21.541716774028956, 0.03493429588503183]),
        array([[ -1.8318173 , -21.59107208],
               [ -1.46293093, -21.52018287],
               [ -1.78116059, -21.58515001],
               ...,
               [ -1.68993735, -21.56580531],
               [ -1.62504524, -21.55827881],
               [ -1.68222206, -21.55991577]])], dtype=object),
 array([list([-2.060977203882005, 0.31780220511632895]),
        list([-21.027675443851365, 0.07464704054331328]),
        array([[ -2.47978154, -21.13003911],
               [ -2.01884781, -21.0341559 ],
               [ -2.43922645, -21.1108629 ],
               ...,
               [ -1.91439503, -20.98474637],
               [ -2.06098604, -21.0347632 ],
               [ -1.81770938, -20.96748075]])], dtype=object),
 array([list([-2.01524040458978, 0.6519294159064167]),
        list([-19.12100567629927, 0.1512393248180146]),
        array([[ -2.15068349, -19.20115706

In [ ]:
np.save("../A-Catalogs/Muv-Beta-EW/slopeMUVJades_zspec.npy",slopeMuvJADES_Zspec)

### Calculate Muv and Beta with 3DHST

Add photometry to catalog
use full phot

Then Run MCMC

In [ ]:
%load_ext autoreload
%autoreload 

import xmatch
full3DHST=ascii.read("../A-Catalogs/goodsn_3dhst.v4.1.cat")
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")


idx,d2d=xmatch.xmatch(Catalog["ra_candels"],Catalog["dec_candels"],full3DHST["ra"],full3DHST["dec"])
mask=[]
for i,d in zip(idx,d2d):
    if d.value<(0.3/3600.):
        mask.append(True)
    else:
        mask.append(False)


Cat3DHST=Catalog[mask]
idx=idx[mask]
d2d=d2d[mask]


Filters=np.array(['f_F160W','e_F160W','f_U','e_U','f_F435W','e_F435W','f_B','e_B','f_G','e_G','f_V','e_V','f_F606W','e_F606W','f_R','e_R','f_Rs','e_Rs','f_I','e_I','f_F775W','e_F775W','f_Z','e_Z','f_F850LP','e_F850LP','f_F125W','e_F125W','f_J','e_J','f_F140W','e_F140W','f_H','e_H',])
for name in Filters:
    Catalog[name] = np.nan

for i in range(0,len(Cat3DHST)):
    IDX=idx[i]
    fluxs=[full3DHST[IDX][f] for f in Filters]
    mags=[]
    for f,n in zip(fluxs,Filters):
        
        if "e" in n:
            magP=Tools.Flux2MagAB_Gabe((prev+f))
            magM=Tools.Flux2MagAB_Gabe((prev-f))
            mags.append(np.mean([magM-prevmag,prevmag-magP]))
        else:
            mags.append(Tools.Flux2MagAB_Gabe(f))
            prev=f
            prevmag=Tools.Flux2MagAB_Gabe(f)

        
    
    
    #mags=[Tools.Flux2MagAB(f*1e-6) for f in fluxs ]
    Data[Cat3DHST[i]["id_charlotte"]]=mags
EmptyArray=[np.nan for gal in Catalog]

for i,name in enumerate(Filters):
    name=name.replace("flux","mag")
    Catalog[name]=EmptyArray
    for gal in Catalog:
        try:
            #print(Data[gal["id_charlotte"]])
            gal[name]=Data[gal["id_charlotte"]][i]
        except:
            continue
            #print("No Jades Data")
ascii.write(Catalog, '../A-Catalogs/Binospec-Candels.cat', overwrite=True)  


In [ ]:
### UPDATE CATALOGS WITH THE NEW COORDINATES, Run all again cuz i fucked f814

#GET MUV BETA AND EW

# AT THE END 15 or 16 sources should have no MUV,beta and EW , do that sanity check

In [ ]:
%load_ext autoreload
%autoreload 

Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")
mask = [gal["id_charlotte"] in highEW for gal in Catalog]
Catalog=Catalog[mask]
PObj= np.array([MCMCObject(c) for c in Catalog])

Cwaves  = [15437.71,4360.06 ,6035.73,7757,9080.26,12503.91,13983.36 ]
EffW    = [[13857.70,17003.09],[3610.23,4883.77],[4634.30,7180.10] ,[6801,8630],[8007.01,10862.13],[10853.22, 14141.73],[11864.94,16133.14]]
#Cwaves  = [15437.71,3582,4360.06 ,4509,4741,5470,6035.73,6283,6789,7697.23,7757,8964,9080.26,12503.91,12525.03,13983.36 ,16364.32]
#EffW    = [[13857.70,17003.09],[3070,4092],[3610.23,4883.77],[3679,5072],[4216,5280],[4841,6178],[4634.30,7180.10] ,[5434,7196],[6012,7673],[6772,8648],[6801,8630],[8131,10096],[8007.01,10862.13],[10853.22, 14141.73],[11527,13539],[11864.94,16133.14],[14722,18029]]



Filters=np.array(['f_F160W','f_F435W','f_F606W','f_F775W','f_F850LP','f_F125W','f_F140W'])
#Filters=np.array(['f_F160W','f_U','f_F435W','f_B','f_G','f_V','f_F606W','f_R','f_Rs','f_I','f_F775W','f_Z','f_F850LP','f_F125W','f_J','f_F140W','f_H'])

FiltNames   =  [name.replace("f","f") for name in Filters]
FiltErrNames=  [name.replace("f","e") for name in Filters]


[gal.setFilters(FiltNames,FiltErrNames) for gal in PObj]
[gal.setCWave(Cwaves) for gal in PObj]
[gal.setEffWidth(EffW) for gal in PObj]
RUN_MCMC=True #Put True if we want to run it
def runCode(p,i=0):
    print(p.cat)
    slopeMuv =   np.array(p.calculateUVslope(ShowPlots=True)) #If dont want plotting, showplots=False
    
    #np.save("SlopeMuv_MCMC.npy",slopeMuv)
    return slopeMuv

#num=[0,"ALL"]
if RUN_MCMC==True:
    # If you want to see the plots, dont run it parallelized
    #slopeMuv3DHST=Parallel(n_jobs=4)(delayed(runCode)(p,i) for i,p in enumerate(PObj[:]))#         Paralellized
    slopeMuvJADES=[runCode(gal) for gal in PObj[:]]      
    #np.save("../A-Catalogs/Muv-Beta-EW/slopeMUV3DHST_Nov1.npy",slopeMuv3DHST)

else:
    slopeMuv3DHST=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUV3DHST_Nov1.npy",allow_pickle=True)

In [ ]:
slopeMuv3DHST=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUV3DHST_Nov1.npy",allow_pickle=True)
slopeMuvJADES=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUVJades_Nov1.npy",allow_pickle=True)
slopeMuvCANDLES=np.load("../A-Catalogs/Muv-Beta-EW/slopeMUVCANDLES.npy",allow_pickle=True)

In [ ]:
from matplotlib.gridspec import GridSpec


LAE=Catalog["Type"]=="LAE"


fig = plt.figure(figsize=(10, 8))
gs = GridSpec(4, 4, figure=fig)

ax_main = fig.add_subplot(gs[1:, :-1])
ax_histx = fig.add_subplot(gs[0, :-1], sharex=ax_main)

# Scatter plot with error bars
#ax_main.errorbar(Catalog["Muv"], Catalog["slope"], xerr=Catalog["dMuv"], yerr=Catalog["dslope"], fmt="o")
ax_main.set_xlabel("Muv")
ax_main.set_ylabel("Beta")
ax_main.invert_xaxis()

# Histogram for Muv
ax_histx.hist(Catalog["Muv"], bins=15, color='gray', edgecolor='black')
ax_histx.set_ylabel('Count')

plt.setp(ax_histx.get_xticklabels(), visible=False)


ax_histy = fig.add_subplot(gs[1:, -1], sharey=ax_main)

# Histogram for slope
ax_histy.hist(Catalog["slope"], bins=15, orientation='horizontal', color='gray', edgecolor='black')
ax_histy.set_xlabel('Count')

plt.setp(ax_histy.get_yticklabels(), visible=False)



# Scatter plot with error bars for LAE points in red
ax_main.errorbar(Catalog["Muv"][LAE], Catalog["slope"][LAE], xerr=Catalog["dMuv"][LAE], yerr=Catalog["dslope"][LAE], fmt="o", color='red', label='LAE',zorder=100)

# Scatter plot with error bars for non-LAE points in blue
ax_main.errorbar(Catalog["Muv"][~LAE], Catalog["slope"][~LAE], xerr=Catalog["dMuv"][~LAE], yerr=Catalog["dslope"][~LAE], fmt="o", color='blue', label='Non-LAE')

ax_main.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
cosmo   =   FlatLambdaCDM(H0=70,Om0=0.3,Tcmb0=2.725)
from astropy.io import ascii


def power_law(chain,Z):
    models=[]
    xs=[]
    for j in range(0,len(chain)):
        z=Z[j]
        temp=[]
        for i in range(0,len(chain[j])):
            c=chain[j]
            Muv=c[i][1]
            beta=c[i][0]
            x=1215.67*(1+z)
            c=2.99792458E+18
            dL      =   cosmo.luminosity_distance(z).value
            DM=5*(np.log10(dL*10**6) - 1)
            K= 2.5*np.log10(1.+z)
            x_1500=1500.*(1+z)
            model=(c/x**2)*((x/x_1500)**(beta+2)) * 10**(-(Muv+48.6+DM-K)/2.5) #flambda
            temp.append(model)
        models.append([np.median(temp),np.std(temp)])
        xs.append(x)
    return xs,models

def GetEW(Lyaflux,dLyaflux,UVflux,z):
    #b,c     =   fitParams[0],fitParams[1]
    #continuumWave=1215.67*(1+z)
    #logflux =   c+b*np.log10(continuumWave)
    #UVflux    =   UVflux*(2.998e18/(continuumWave**2))
    dUVflux=UVflux[1]
    UVflux=UVflux[0]
    Lyaflux=10**Lyaflux
    dLyaflux=10**dLyaflux
    EW      =   (Lyaflux)/UVflux
    EWrest=EW/(1+z)
    dEW=EWrest*np.sqrt((dUVflux/UVflux)**2+(dLyaflux/Lyaflux)**2)
    return EWrest,dEW

def Get_fv(Muv,z):
    cosmo   =   FlatLambdaCDM(H0=70,Om0=0.3,Tcmb0=2.725)
    dL      =   cosmo.luminosity_distance(z).value
    m    =   Muv+5*(np.log10(dL*10**6) - 1)
    flux    =   10**(-0.4*(m+48.6))
    flux=flux
    c=3.0e18
    flux=flux*(c/(1500)**2)
    return flux

In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")

Muv=np.array(Catalog['Muv'])
dMuv=np.array(Catalog['dMuv'])
beta=np.array(Catalog['slope'])
dbeta=np.array(Catalog['dslope'])
EW=np.array(Catalog['EWLya'])
Type=Catalog["Type"]=="LAE"
z=[]
for gal in Catalog:
    if gal["zsys"]>0:
        z.append(gal['zsys'])
    elif gal["z"]>0:
        z.append(gal['z'])
    elif gal["photoz"]>0:
        z.append(gal['photoz'])
z=np.array(z)

dLya_non_detect={}

for gal in Catalog:
    if np.isnan(gal["LyaFlux"])==True:
        #print(gal["id_charlotte"])
        flya=10**gal["FluxLim"]
        try:
            fl=np.load("/Users/gonzalo/Desktop/Code/Gonzalo_Binospec/A-Project/A-Catalogs/data/large_files/LowLim/"+gal["id_charlotte"]+".npy")
        except:
            print(gal["id_charlotte"])
            dLya_non_detect[gal["id_charlotte"]]=6e-18
            continue
        err=np.nanstd(fl)
        constant=flya/np.nanmedian(fl)

        flya=flya
        flya_err=err
        
        dLya_non_detect[gal["id_charlotte"]]=err



flya=[]
dflya=[]
for gal in Catalog:
    if gal["LyaFlux"]<0:
        flya.append(gal["LyaFlux"])
        dflya.append(gal["LyaErr"])
    elif gal["FluxLim"]<0:
        flya.append(gal["FluxLim"])
        try:
            dflya.append(np.log10(dLya_non_detect[gal["id_charlotte"]]))
        except:
            dflya.append(np.log10(6e-18))
    else:
        flya.append(np.nan)
        dflya.append(np.nan)
flya=np.array(flya)
dflya=np.array(dflya)





### Plotting EW old and new vs slope and Muv

Lets see if its a problem of JADES or 3DHST?

In [ ]:
def mask_both(cat1,cat2):
    mask_both=[]
    for i in range(0,len(cat1)):
        if np.isnan(cat1[i][1][0])==False and np.isnan(cat2[i][1][0])==False:
            mask_both.append(True)
        else:
            mask_both.append(False)
    return mask_both

In [ ]:
slopes3DHST=np.array([slopeMuv3DHST[i][0][0] for i in range(0,len(slopeMuv3DHST))])
Muv3DHST=np.array([slopeMuv3DHST[i][1][0] for i in range(0,len(slopeMuv3DHST))])
slopesJades=np.array([slopeMuvJADES[i][0][0] for i in range(0,len(slopeMuvJADES))])
MuvJades=np.array([slopeMuvJADES[i][1][0] for i in range(0,len(slopeMuvJADES))])
slopesCANDLES=np.array([slopeMuvCANDLES[i][0][0] for i in range(0,len(slopeMuvCANDLES))])
MuvCANDLES=np.array([slopeMuvCANDLES[i][1][0] for i in range(0,len(slopeMuvCANDLES))])

chains3DHST=np.array([slopeMuv3DHST[i][2] for i in range(0,len(slopeMuv3DHST))])
chainsJades=np.array([slopeMuvJADES[i][2] for i in range(0,len(slopeMuvJADES))])
chainsCANDLES=np.array([slopeMuvCANDLES[i][2] for i in range(0,len(slopeMuvCANDLES))])

x,pL=power_law(chains3DHST,z)
pL=np.array(pL)
EW_3DHST=[]
for LYA,dLya,UV,Z in zip(flya,dflya,pL,z):
    EW_3DHST.append(GetEW(LYA,dLya,UV,Z))
EW_3DHST=np.array(EW_3DHST)


x,pL=power_law(chainsJades,z)
EW_Jades=[]
for LYA,dLya,UV,Z in zip(flya,dflya,pL,z):
    EW_Jades.append(GetEW(LYA,dLya,UV,Z))
EW_Jades=np.array(EW_Jades)


x,pL=power_law(chainsCANDLES,z)
EW_CANDLES=[]
for LYA,dLya,UV,Z in zip(flya,dflya,pL,z):
    EW_CANDLES.append(GetEW(LYA,dLya,UV,Z))
EW_CANDLES=np.array(EW_CANDLES)


In [ ]:
plt.errorbar(EW_3DHST[:,0],EW_Jades[:,0],xerr=EW_3DHST[:,1],yerr=EW_Jades[:,1],fmt="o",alpha=0.4)
plt.plot([0,200],[0,200],ls="--")
plt.xlim(0,300)
plt.ylim(0,300)
plt.xlabel("EW 3DHST")
plt.ylabel("EW Jades")
plt.show()

plt.errorbar(EW_3DHST[:,0],EW_CANDLES[:,0],xerr=EW_3DHST[:,1],yerr=EW_CANDLES[:,1],fmt="o",alpha=0.4)
plt.plot([0,200],[0,200],ls="--")
plt.xlim(0,300)
plt.ylim(0,300)
plt.xlabel("EW 3DHST")
plt.ylabel("EW CANDLES")
plt.show()

plt.errorbar(EW_Jades[:,0],EW_CANDLES[:,0],xerr=EW_Jades[:,1],yerr=EW_CANDLES[:,1],fmt="o",alpha=0.4)
plt.plot([0,200],[0,200],ls="--")
plt.xlim(0,300)
plt.ylim(0,300)
plt.xlabel("EW Jades")
plt.ylabel("EW CANDLES")
plt.show()


In [ ]:
Red=['z6_GND_8934',
 'z5_GND_10574',
 'z6_GNW_11480',
 'z7_GNW_22375',
 'z6_GND_43953',
 'z5_GNW_29609',
 'z5_GND_6005',
 'z7_GNW_7284']

Red=Red+["z6_GND_34516","z5_GND_908"]+["z5_GNW_23799","Jung18_z6_GND_28438"]

Red_3DHST=["z7_GNW_22375","z5_GNW_29609","z6_GND_29625"]
 

In [ ]:
Catalog=ascii.read("../A-Catalogs/Binospec-Candels.cat")
reWrite=True
maskJades=[False if np.isnan(gal[1][0])==True else True for gal in slopeMuvJADES]
mask_CANDLES=[False if np.isnan(gal[1][0])==True else True for gal in slopeMuvCANDLES]
Muv,dMuv=[],[]
EWLya=[]
dEWLya=[]
slope,dslope=[],[]
if reWrite==True:
    for i,gal in enumerate(Catalog):

        if gal["id_charlotte"] in Red_3DHST:
            Muv.append(slopeMuv3DHST[i][1][0])
            dMuv.append(slopeMuv3DHST[i][1][1])
            slope.append(slopeMuv3DHST[i][0][0])
            dslope.append(slopeMuv3DHST[i][0][1])
            EWLya.append(EW_3DHST[i][0])
            dEWLya.append(EW_3DHST[i][1])
            continue

        if gal["id_charlotte"] in Red:
            Muv.append(slopeMuvCANDLES[i][1][0])
            dMuv.append(slopeMuvCANDLES[i][1][1])
            slope.append(slopeMuvCANDLES[i][0][0])
            dslope.append(slopeMuvCANDLES[i][0][1])
            EWLya.append(EW_CANDLES[i][0])
            dEWLya.append(EW_CANDLES[i][1])
            continue       

        if gal["id_charlotte"]=="z5_GND_10574":
            Muv.append(slopeMuvCANDLES[i][1][0])
            dMuv.append(slopeMuvCANDLES[i][1][1])
            slope.append(slopeMuvCANDLES[i][0][0])
            dslope.append(slopeMuvCANDLES[i][0][1])
            EWLya.append(EW_CANDLES[i][0])
            dEWLya.append(EW_CANDLES[i][1])
            continue

        if np.isnan(slopeMuvJADES[i][1][0])==False:
            Muv.append(slopeMuvJADES[i][1][0])
            dMuv.append(slopeMuvJADES[i][1][1])
            slope.append(slopeMuvJADES[i][0][0])
            dslope.append(slopeMuvJADES[i][0][1])
            EWLya.append(EW_Jades[i][0])
            dEWLya.append(EW_Jades[i][1])
        elif np.isnan(slopeMuvCANDLES[i][1][0])==False:
            Muv.append(slopeMuvCANDLES[i][1][0])
            dMuv.append(slopeMuvCANDLES[i][1][1])
            slope.append(slopeMuvCANDLES[i][0][0])
            dslope.append(slopeMuvCANDLES[i][0][1])
            EWLya.append(EW_CANDLES[i][0])
            dEWLya.append(EW_CANDLES[i][1])
        elif np.isnan(slopeMuv3DHST[i][1][0])==False:
            Muv.append(slopeMuv3DHST[i][1][0])
            dMuv.append(slopeMuv3DHST[i][1][1])
            slope.append(slopeMuv3DHST[i][0][0])
            dslope.append(slopeMuv3DHST[i][0][1])
            EWLya.append(EW_3DHST[i][0])
            dEWLya.append(EW_3DHST[i][1])
        else:
            Muv.append(np.nan)
            dMuv.append(np.nan)
            slope.append(np.nan)
            dslope.append(np.nan)
            EWLya.append(np.nan)
            dEWLya.append(np.nan)


    Catalog["Muv"]=Muv
    Catalog["dMuv"]=dMuv
    Catalog["slope"]=slope
    Catalog["dslope"]=dslope
    Catalog["EWLya"]=EWLya
    Catalog["dEWLya"]=dEWLya
    ascii.write(Catalog, '../A-Catalogs/Binospec-Candels.cat', overwrite=True)

In [ ]:
plt.scatter(Catalog["slope"],Catalog["EWLya"])

## CANDLES/SHARDS HST

In [ ]:
for gal in Catalog:
    if gal["id_charlotte"]=="z5_GND_10574":
        print(gal["Muv"],gal["dMuv"],gal["slope"],gal["dslope"],gal["EWLya"],gal["dEWLya"])

In [ ]:
plt.scatter(Catalog["EWLya"],Catalog["slope"])

In [ ]:
i=0
for gal in Catalog:
    if np.isnan(gal["Muv"]):
        print(gal["Type"],gal["id_charlotte"])
        i+=1
print(i)

In [ ]:
s1=[i[0] for i in slopeMuvCANDLES[:,0]]
s2=[i[0] for i in slopeMuvJADES[:,0]]
s3=[i[0] for i in slopeMuv3DHST[:,0]]

In [ ]:
plt.scatter(s1,s2)
plt.plot([-5,1],[-5,1],ls="--")
plt.show()

plt.scatter(s3,s2)
plt.plot([-5,1],[-5,1],ls="--")

In [ ]:
slopes3DHST=np.array([slopeMuv3DHST[i][0][0] for i in range(0,len(slopeMuv3DHST))])
Muv3DHST=np.array([slopeMuv3DHST[i][1][0] for i in range(0,len(slopeMuv3DHST))])
slopesJades=np.array([slopeMuvJADES[i][0][0] for i in range(0,len(slopeMuvJADES))])
MuvJades=np.array([slopeMuvJADES[i][1][0] for i in range(0,len(slopeMuvJADES))])
slopesCANDLES=np.array([slopeMuvCANDLES[i][0][0] for i in range(0,len(slopeMuvCANDLES))])
MuvCANDLES=np.array([slopeMuvCANDLES[i][1][0] for i in range(0,len(slopeMuvCANDLES))])
EW_3DHST=np.array([EW_3DHST[i][0] for i in range(0,len(EW_3DHST))])
EW_Jades=np.array([EW_Jades[i][0] for i in range(0,len(EW_Jades))])
EW_CANDLES=np.array([EW_CANDLES[i][0] for i in range(0,len(EW_CANDLES))])


In [ ]:


mask_Both=mask_both(slopeMuv3DHST,slopeMuvJADES)
plt.scatter(slopes3DHST[mask_Both], slopesJades[mask_Both], c=EW_3DHST[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta 3DHST")
plt.ylabel("Beta JADES")
plt.show()

plt.scatter(slopes3DHST[np.logical_and(mask_Both, Type)], slopesJades[np.logical_and(mask_Both, Type)], c=EW_3DHST[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta 3DHST")
plt.ylabel("Beta JADES")
plt.title("LAE only")
plt.show()

plt.scatter(Muv3DHST[mask_Both], MuvJades[mask_Both], c=EW_3DHST[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv 3DHST")
plt.ylabel("Muv JADES")
plt.show()

plt.scatter(Muv3DHST[np.logical_and(mask_Both, Type)], MuvJades[np.logical_and(mask_Both, Type)], c=EW_3DHST[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv 3DHST")
plt.ylabel("Muv JADES")
plt.title("LAE only")
plt.show()


mask_Both=mask_both(slopeMuvCANDLES,slopeMuvJADES)
plt.scatter(slopesCANDLES[mask_Both], slopesJades[mask_Both], c=EW_CANDLES[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW CANDLES')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta CANDLES")
plt.ylabel("Beta JADES")
plt.show()

plt.scatter(slopesCANDLES[np.logical_and(mask_Both, Type)], slopesJades[np.logical_and(mask_Both, Type)], c=EW_CANDLES[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW CANDLES')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta CANDLES")
plt.ylabel("Beta JADES")
plt.title("LAE only")
plt.show()

plt.scatter(MuvCANDLES[mask_Both], MuvJades[mask_Both], c=EW_CANDLES[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW CANDLES')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv CANDLES")
plt.ylabel("Muv JADES")
plt.show()

plt.scatter(MuvCANDLES[np.logical_and(mask_Both, Type)], MuvJades[np.logical_and(mask_Both, Type)], c=EW_CANDLES[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW CANDLES')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv CANDLES")
plt.ylabel("Muv JADES")
plt.title("LAE only")
plt.show()


mask_Both=mask_both(slopeMuv3DHST,slopeMuvCANDLES)
plt.scatter(slopes3DHST[mask_Both], slopesCANDLES[mask_Both], c=EW_3DHST[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.colorbar(label='EW 3DHST')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta 3DHST")
plt.ylabel("Beta CANDLES")
plt.show()

plt.scatter(slopes3DHST[np.logical_and(mask_Both, Type)], slopesCANDLES[np.logical_and(mask_Both, Type)], c=EW_3DHST[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-4, 1], [-4, 1], ls="--")
plt.xlabel("Beta 3DHST")
plt.ylabel("Beta CANDLES")
plt.title("LAE only")
plt.show()

plt.scatter(Muv3DHST[mask_Both], MuvCANDLES[mask_Both], c=EW_3DHST[mask_Both], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv 3DHST")
plt.ylabel("Muv CANDLES")
plt.show()

plt.scatter(Muv3DHST[np.logical_and(mask_Both, Type)], MuvCANDLES[np.logical_and(mask_Both, Type)], c=EW_3DHST[np.logical_and(mask_Both, Type)], cmap='viridis', vmin=0, vmax=150)
plt.colorbar(label='EW 3DHST')
plt.plot([-24, -18], [-24, -18], ls="--")
plt.xlabel("Muv 3DHST")
plt.ylabel("Muv CANDLES")
plt.title("LAE only")
plt.show()





In [ ]:
sc = plt.scatter(EW, newEW, c=Muv, cmap='viridis',vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0,200],[0,200],color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.xlim(0,200)
plt.ylim(0,200)
plt.show()


sc = plt.scatter(EW, newEW, c=beta, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0,200],[0,200],color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.xlim(0,200)
plt.ylim(0,200)
plt.show()


####For Jades

EW_true = EW[maskJades]
newEW_true = newEW[maskJades]
Muv_true = Muv[maskJades]
beta_true = beta[maskJades]

sc = plt.scatter(EW_true, newEW_true, c=Muv_true, cmap='viridis', vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW JADES")
plt.title("JADES only")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

sc = plt.scatter(EW_true, newEW_true, c=beta_true, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW JADES")
plt.title("JADES only")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

# Jades Only but also use Type mask
EW_true = EW[np.logical_and(maskJades, Type)]
newEW_true = newEW[np.logical_and(maskJades, Type)]
Muv_true = Muv[np.logical_and(maskJades, Type)]
beta_true = beta[np.logical_and(maskJades, Type)]

# First plot with Muv as color
sc = plt.scatter(EW_true, newEW_true, c=Muv_true, cmap='viridis', vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW JADES")
plt.title("JADES only (LAE mask)")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

# Second plot with beta as color
sc = plt.scatter(EW_true, newEW_true, c=beta_true, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW JADES")
plt.title("JADES only (LAE mask)")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()


###For 3DHST


EW_true = EW
newEW_true = EW_3DHST
Muv_true = Muv3DHST
beta_true = slopes3DHST

sc = plt.scatter(EW_true, newEW_true, c=Muv_true, cmap='viridis', vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST ")
plt.title("3DHST only")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

sc = plt.scatter(EW_true, newEW_true, c=beta_true, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.title("3DHST only")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

EW_true = EW[Type]
newEW_true = EW_3DHST[Type]
Muv_true = Muv3DHST[Type]
beta_true = slopes3DHST[Type]

sc = plt.scatter(EW_true, newEW_true, c=Muv_true, cmap='viridis', vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST ")
plt.title("3DHST only (LAE only)")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

sc = plt.scatter(EW_true, newEW_true, c=beta_true, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.title("3DHST only (LAE only)")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()




### FOR LAES
# Filter data for elements where Type is True
EW_true = EW[Type]
newEW_true = newEW[Type]
Muv_true = Muv[Type]
beta_true = beta[Type]

# First plot with Muv as color
sc = plt.scatter(EW_true, newEW_true, c=Muv_true, cmap='viridis', vmin=-22, vmax=-19)
plt.colorbar(sc, label='Muv')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()

# Second plot with beta as color
sc = plt.scatter(EW_true, newEW_true, c=beta_true, cmap='viridis', vmin=-3, vmax=-0.5)
plt.colorbar(sc, label='slope')
plt.plot([0, 200], [0, 200], color='red')
plt.xlabel("EW old catalog")
plt.ylabel("EW 3DHST or JADES")
plt.xlim(0, 200)
plt.ylim(0, 200)
plt.show()



In [ ]:

# Plot for LAE with red circles
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 18})


plt.errorbar(beta[Type], EW[Type], xerr=dbeta[Type], fmt='o', color='red', label='LAE',zorder=100,alpha=0.8)
# Plot for non-LAE with grey arrows pointing down
plt.errorbar(beta[~Type], EW[~Type], xerr=dbeta[~Type], fmt='v', color='gray', label='Non-LAE',alpha=0.5)

plt.xlabel("Slope")
plt.ylabel("EW old")
plt.ylim(-20,420)
plt.legend()
plt.show()

plt.errorbar(beta[Type], newEW[Type], xerr=dbeta[Type], fmt='o', color='red', label='LAE',zorder=100,alpha=0.8)
# Plot for non-LAE with grey arrows pointing down
plt.errorbar(beta[~Type], newEW[~Type], xerr=dbeta[~Type], fmt='v', color='gray', label='Non-LAE',alpha=0.5)

plt.xlabel("Slope")
plt.ylabel("EW new")
plt.ylim(-20,420)
plt.legend()
plt.show()



plt.errorbar(Muv[Type], EW[Type], xerr=dMuv[Type], fmt='o', color='red', label='LAE',zorder=100,alpha=0.8)
# Plot for non-LAE with grey arrows pointing down
plt.errorbar(Muv[~Type], EW[~Type], xerr=dMuv[~Type], fmt='v', color='gray', label='Non-LAE',alpha=0.5)

plt.xlabel("Muv")
plt.ylabel("EW old")
plt.ylim(-20,420)
plt.legend()
plt.show()

plt.errorbar(Muv[Type], newEW[Type], xerr=dMuv[Type], fmt='o', color='red', label='LAE',zorder=100,alpha=0.8)
# Plot for non-LAE with grey arrows pointing down
plt.errorbar(Muv[~Type], newEW[~Type], xerr=dMuv[~Type], fmt='v', color='gray', label='Non-LAE',alpha=0.5)

plt.xlabel("Muv")
plt.ylabel("EW new")
plt.ylim(-20,420)
plt.legend()
plt.show()

In [ ]:
plt.hist(EW-newEW,bins=20,range=(-100,100))